In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tifffile as tiff

Manual input parameters

In [3]:
expected_neuron_radius_pix = 5
dff_baseline_size = 300
dff_baseline_percent = 75
min_pk_height = 0.2

Load image stack 


In [3]:
path = '../data/DFFtest.tif'
img = tiff.imread(path) # array of images: use [index] to access a specific slide
img_array = np.array(img) # np array of coordinates/values
# 1st dimension: image, 2nd dimension: row of image, 3rd dimension: col of image
# plt.imshow(img[420]) # 420th img in the tif
# print(img_array[0 : 5])
img_array = img_array / 255
print(img_array[0])

[[0.00392157 0.00392157 0.00784314 ... 0.00392157 0.00392157 0.00392157]
 [0.00392157 0.00784314 0.00392157 ... 0.00392157 0.00392157 0.        ]
 [0.00784314 0.00392157 0.00392157 ... 0.00392157 0.00392157 0.00392157]
 ...
 [0.0745098  0.07058824 0.07058824 ... 0.01176471 0.00784314 0.01176471]
 [0.06666667 0.07058824 0.07058824 ... 0.01176471 0.00392157 0.01176471]
 [0.06666667 0.07058824 0.07843137 ... 0.00784314 0.00784314 0.00784314]]


In [9]:
mean_img = (np.mean(img_array, axis = 0))
print(mean_img[0, : 20])

[0.00556471 0.00592549 0.00640392 0.00632549 0.00698824 0.00669804
 0.00599608 0.00672157 0.00741176 0.00575686 0.00696078 0.00857255
 0.00700784 0.00769804 0.00896863 0.00740784 0.00839216 0.00723529
 0.00809412 0.00816863]


In [3]:
std_img = (np.std(img_array, axis = 0))
print(std_img[0, : 2])

NameError: name 'img_array' is not defined

Measure variance of image

Identify cell centers

In [ ]:
# manual

In [ ]:
# load

In [ ]:
# automated (difference of gaussian)

Extract fluorescence over time from a circle around selected cell coordinates

In [ ]:
# FUNCTION: fluo extraction (assume all ROI are the same)

dF/F calculation

In [ ]:
# FUNCTION: sliding window dF/f calculation

Look for peaks (test)

Correlation coefficient (test)

Cross correlation (test)

Distance matrix

Plot dF/F and correlation matrix

Get triangles

Plot distance v correlation scatters

In [ ]:
# FUNCTION: binned mean

In [ ]:
# FUNCTION: Scatter kernel density estimate

In [ ]:
# kernel density estimate

In [ ]:
# desu density scattercloud

Histogram of correlation coefficient